In [2]:
import sys
sys.path.insert(0, '/home/sahre/git_repositories/atomic_force_field/')

import alchemical_decomposition as ad

sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')

import utils_qm as uqm

from parse_density_files import CUBE

import os
import numpy as np

###########################################################
#  gather data for compound
###########################################################

# contains the data necessary to calculate atomic energies
# is used to initialize an AlchemicalMolecule object
datadict = dict()

# get lambda values
n_ve = 32
lam_vals = np.array([0, 13, 19, 26, 32])/n_ve
datadict['lam_vals'] = lam_vals

# get density grid
density_grid = np.load('/data/sahre/projects/atomic_force_field/parameters/density_grid_butanol.npy')
datadict['density_grid'] = density_grid

# get hmatrix
hmatrix =np.load('/data/sahre/projects/atomic_force_field/parameters/h_matrix_butanol.npy')
datadict['hmatrix'] = hmatrix

# read molecule specific information (densities, charge and position of nuclei) necessary to calculate atomic energies
compound_path = '/data/sahre/projects/atomic_force_field/bond_stretch/butanol/dist_0.0'

cube_files = ['ve_13.cube', 've_19.cube', 've_26.cube', 've_32.cube']

densities = []
nuclei = []
for cube_file in cube_files:
    cube_path = compound_path + '/cube-files/' + cube_file
    
    # needed because some cube-files are named ve_8.cube instead of ve_08.cube
    if cube_file == 've_08.cube':
        if not os.path.isfile(cube_path):
            cube_path = cube_path.replace('ve_08', 've_8')
        
    assert os.path.isfile(cube_path), f'{cube_path} does not exist'
    
    cube_object = CUBE(cube_path)
    
    N = n_ve/cube_object.data_scaled.sum()
    
    densities.append(N*cube_object.data_scaled)
    
    nuclei = cube_object.atoms

# add the density of the uniform electron gas
num_gpt = densities[0].shape[0]*densities[0].shape[1]*densities[0].shape[2]
num_ve = densities[-1].sum()
ueg = np.full_like(densities[0], fill_value=num_ve/num_gpt)
densities.insert(0, ueg)
datadict['densities'] = densities

# initialize alchemical atoms

nuclear_charges = nuclei[:,0]
positions = nuclei[:, 1:4]
AlchAtoms = []
pp_parameters = uqm.load_obj('/data/sahre/projects/atomic-energies/parameters/pp_parameters.dict')
for Z, position in zip(nuclear_charges, positions):
    AlchAtoms.append(ad.AlchemicalAtom({'Z':Z, "position":position, "pp_parameters":pp_parameters[Z]}))
datadict['AlchAtoms'] = AlchAtoms

###########################################################
# create alchemical molecule
###########################################################

alchMol = ad.AlchemicalMolecule(datadict)

###########################################################
# calculate atomic energy
###########################################################

alchMol.calculate_atomic_energies(lambda_intg='trapz', potential='pp') # pseudopotential
energies_pp = alchMol.atomic_energies.copy()

alchMol.calculate_atomic_energies(lambda_intg='trapz', potential='coulomb_ZV') # coolomb potential
energies_coul = alchMol.atomic_energies.copy()

###########################################################
# save atomic energy data
###########################################################

store = np.array([nuclei[:,0], alchMol.get_ZV(), nuclei[:,1], nuclei[:,2], nuclei[:,3], energies_pp, energies_coul]).T
header = 'nuclear charge\t nuclear valence charge\t x_coord\t y_coord\t z_coord\t atomic energy pp\t atomic energy coulomb\t'
save_dir = os.path.join(compound_path, 'atomic_energies_pp.txt')
np.savetxt(save_dir, store, delimiter='\t', header = header)